In [1]:
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from connect import connect_sqlalchemy
from config import load_config

In [2]:
config = load_config()
engine = connect_sqlalchemy(config)

Connected to PostgreSQL using SQLAlchemy.


In [3]:
query = """WITH UNION_FLIGHTS AS
(SELECT FLIGHT_ID,
		DEPARTURE_AIRPORT AS AIRPORT_CODE,
		ACTUAL_DEPARTURE AS DATE_OF_FLIGHT_IN_AIRPORT
	FROM FLIGHTS
	WHERE ACTUAL_DEPARTURE IS NOT NULL
	UNION SELECT FLIGHT_ID,
		ARRIVAL_AIRPORT AS AIRPORT_CODE,
		ACTUAL_ARRIVAL AS DATE_OF_FLIGHT_IN_AIRPORT
	FROM FLIGHTS
	WHERE ACTUAL_ARRIVAL IS NOT NULL )
SELECT AD.AIRPORT_NAME ->> 'en' AS AIRPORT_NAME,
	TO_CHAR(UF.DATE_OF_FLIGHT_IN_AIRPORT,'Day') AS DAY_NAME,
	COUNT(*) AS COUNT_FLIGHTS
FROM UNION_FLIGHTS UF
JOIN AIRPORTS_DATA AD ON UF.AIRPORT_CODE = AD.AIRPORT_CODE
WHERE UF.DATE_OF_FLIGHT_IN_AIRPORT <
		(SELECT MIN(DATE_OF_FLIGHT_IN_AIRPORT)
			FROM UNION_FLIGHTS) + INTERVAL '1 year'
GROUP BY AD.AIRPORT_NAME,
	TO_CHAR(UF.DATE_OF_FLIGHT_IN_AIRPORT, 'Day')
ORDER BY AD.AIRPORT_NAME,
CASE TRIM(TO_CHAR(UF.DATE_OF_FLIGHT_IN_AIRPORT, 'Day'))
	WHEN 'Monday' THEN 1
	WHEN 'Tuesday' THEN 2
	WHEN 'Wednesday' THEN 3
	WHEN 'Thursday' THEN 4
	WHEN 'Friday' THEN 5
	WHEN 'Saturday' THEN 6
	ELSE 7
END;"""

In [4]:
# Fetch data into a DataFrame 
if engine:
    df = pd.read_sql(query, engine)

df.head()

,airport_name,day_name,count_flights
0,Abakan Airport,Monday,371
1,Abakan Airport,Tuesday,468
2,Abakan Airport,Wednesday,416
3,Abakan Airport,Thursday,364
4,Abakan Airport,Friday,312
